In [2]:
%reset -f

In [3]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [4]:
import re
import string
import numpy as np
import pandas as pd
import nltk
from nltk.corpus import twitter_samples 

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords 
stop_words = set(stopwords.words('english'))

nltk.download('twitter_samples')

[nltk_data] Downloading package twitter_samples to /root/nltk_data...
[nltk_data]   Unzipping corpora/twitter_samples.zip.


True

In [5]:
all_positive_tweets = twitter_samples.strings('positive_tweets.json')
all_negative_tweets = twitter_samples.strings('negative_tweets.json')

In [6]:
test_pos = all_positive_tweets[4000:]
train_pos = all_positive_tweets[:4000]
test_neg = all_negative_tweets[4000:]
train_neg = all_negative_tweets[:4000]

train_x = train_pos + train_neg 
test_x = test_pos + test_neg

# combine positive and negative labels
train_y = np.append(np.ones((len(train_pos), 1)), np.zeros((len(train_neg), 1)), axis=0)
test_y = np.append(np.ones((len(test_pos), 1)), np.zeros((len(test_neg), 1)), axis=0)

In [7]:
# Function to tokenize and preprocess the data
def preprocess_data(line):
    line = re.sub(r'#','',line)
    line = re.sub(r'@','',line)
    clean_line = [word for word in word_tokenize(line) if word not in string.punctuation and word not in stop_words]
    clean_line= "".join(clean_line)
    return clean_line

In [8]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [9]:
train_x = [preprocess_data(line) for line in train_x]
test_x = [preprocess_data(line) for line in test_x]


In [10]:
# Try tf-idf and logistic regression
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer=TfidfVectorizer(stop_words='english',max_features=128)
docs_tfidf=tfidf_vectorizer.fit_transform(train_x)


In [11]:
from sklearn.linear_model import LogisticRegression
clf=LogisticRegression()
clf.fit(docs_tfidf,train_y)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [12]:
from sklearn.metrics import mean_squared_error
lin_reg_pred=clf.predict(docs_tfidf)
lin_reg_mse = mean_squared_error(train_y,lin_reg_pred)
lin_reg_mse

0.46075

**underfitting**

In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer=TfidfVectorizer(stop_words='english',max_features=128)
docs_tfidf=tfidf_vectorizer.fit_transform(test_x)


In [14]:
lin_reg_pred=clf.predict(docs_tfidf)
lin_reg_mse = mean_squared_error(test_y,lin_reg_pred)
lin_reg_mse

0.499

In [18]:
from sklearn.metrics import accuracy_score
pred_train=clf.predict(docs_tfidf)
pred_train_data=pred_train.reshape((288,1))
print('accuracy_score on training data: ',accuracy_score(train_y,pred_train_data))

ValueError: ignored